# Lab Assignment 8: Data Management Using `pandas`, Part 1
## DS 6001: Practice and Application of Data Science

- Name: Congxin (David) Xu
- Computing ID: cx2rx

### Instructions
Please answer the following questions as completely as possible using text, code, and the results of code as needed. Format your answers in a Jupyter notebook. To receive full credit, make sure you address every part of the problem, and make sure your document is formatted in a clean and professional way.

In this lab, you will be working with the [2017 Workplace Health in America survey](https://www.cdc.gov/workplacehealthpromotion/survey/data.html) which was conducted by the Centers for Disease Control and Prevention. According to the survey's [guidence document](https://www.cdc.gov/workplacehealthpromotion/data-surveillance/docs/2017-WHA-Guidance-Document-for-Use-of-Public-Data-files-508.pdf):

> The Workplace Health in America (WHA) Survey gathered information from a cross-sectional, nationally representative sample of US worksites. The sample was drawn from the Dun & Bradstreet (D&B) database of all private and public employers in the United States with at least 10 employees. Like previous national surveys, the worksite served as the sampling unit rather than the companies or firms to which the worksites belonged. Worksites were selected using a stratified simple random sample (SRS) design, where the primary strata were ten multi-state regions defined by the Centers for Disease Control and Prevention (CDC), plus an additional stratum containing all hospital worksites. 

The data contain over 300 features that report the industry and type of company where the respondents are employed, what kind of health insurance and other health programs are offered, and other characteristics of the workplaces including whether employees are allowed to work from home and the gender and age makeup of the workforce. The data are full of interesting information, but in order to make use of the data a great deal of data manipulation is required first.

## Problem 0
Import the following libraries:

In [1]:
import numpy as np
import pandas as pd
import sidetable
import sqlite3
import warnings
warnings.filterwarnings('ignore')

## Problem 1
The raw data are stored in an ASCII file on the 2017 Workplace Health in America survey [homepage](https://www.cdc.gov/workplacehealthpromotion/survey/data.html). Load the raw data directly into Python without downloading the data onto your harddrive and display a dataframe with only the 14th, 28th, and 102nd rows of the data. [1 point]

In [2]:
data = pd.read_csv("https://www.cdc.gov/workplacehealthpromotion/data-surveillance/docs/whpps_120717.csv",
                   sep = "~")

data.iloc[[14, 28, 102],]

,OC1,OC3,HI1,HI2,HI3,HI4,HRA1,HRA1A,HRA1B,HRA1E,...,WL3_05,E1_09,Suppquex,Id,Region,CDC_Region,Industry,Size,Varstrata,"Finalwt_worksite,,,,"
14,7,2.0,2.0,1.0,2.0,1.0,1.0,3.0,2.0,2.0,...,NaN,NaN,2.0,1539.0,2.0,4.0,7.0,5.0,0.0,"47.793940929,,,,"
28,1,3.0,2.0,3.0,1.0,1.0,2.0,96.0,96.0,96.0,...,NaN,NaN,2.0,2755.0,3.0,5.0,7.0,6.0,0.0,"47.793940929,,,,"
102,1,3.0,2.0,3.0,1.0,1.0,1.0,1.0,4.0,2.0,...,NaN,NaN,2.0,12686.0,3.0,5.0,7.0,8.0,0.0,"47.793940929,,,,"


In [13]:
data.Industry.describe()

count    2842.000000
mean        3.804715
std         1.959338
min         1.000000
25%         2.000000
50%         4.000000
75%         5.000000
max         7.000000
Name: Industry, dtype: float64

## Problem 2 
The data contain 301 columns. Create a new variable in Python's memory to store a working version of the data. In the working version, delete all of the columns except for the following:

* `Industry`: 7 Industry Categories with NAICS codes

* `Size`: 8 Employee Size Categories

* `OC3` Is your organization for profit, non-profit, government?

* `HI1` In general, do you offer full, partial or no payment of premiums for personal health insurance for full-time employees?

* `HI2` Over the past 12 months, were full-time employees asked to pay a larger proportion, smaller proportion or the same proportion of personal health insurance premiums?

* `HI3`: Does your organization offer personal health insurance for your part-time employees?

* `CP1`: Are there health education programs, which focus on skill development and lifestyle behavior change along with information dissemination and awareness building?

* `WL6`: Allow employees to work from home?

* Every column that begins `WD`, expressing the percentage of employees that have certain characteristics at the firm

[1 point]

In [4]:
mycols = ['Industry', 'Size', 'OC3', 'HI1', 'HI2', 'HI3', 'CP1', 'WL6'] + \
         list(data.columns[data.columns.str.startswith('WD')])
data_wv = data[mycols]
data_wv.head()

,Industry,Size,OC3,HI1,HI2,HI3,CP1,WL6,WD1_1,WD1_2,WD2,WD3,WD4,WD5,WD6,WD7
0,7.0,7.0,3.0,2.0,1.0,2.0,1.0,1.0,25.0,20.0,85.0,60.0,40.0,15.0,0.0,22.0
1,7.0,6.0,3.0,2.0,3.0,1.0,1.0,1.0,997.0,997.0,90.0,90.0,997.0,997.0,0.0,997.0
2,7.0,8.0,3.0,1.0,3.0,1.0,1.0,1.0,35.0,4.0,997.0,997.0,40.0,15.0,997.0,997.0
3,7.0,4.0,2.0,1.0,2.0,1.0,2.0,2.0,50.0,15.0,50.0,85.0,75.0,0.0,0.0,997.0
4,7.0,4.0,3.0,1.0,3.0,1.0,1.0,1.0,50.0,40.0,60.0,60.0,40.0,30.0,0.0,28.0


In [5]:
data_wv.Industry.describe()

count    2842.000000
mean        3.804715
std         1.959338
min         1.000000
25%         2.000000
50%         4.000000
75%         5.000000
max         7.000000
Name: Industry, dtype: float64

## Problem 3
The [codebook](https://www.cdc.gov/workplacehealthpromotion/data-surveillance/docs/2017-WHA-Datafile-Codebook-508.pdf) for the WHA data contain short descriptions of the meaning of each of the columns in the data. Use these descriptions to decide on better and more intuitive names for the columns in the working version of the data, and rename the columns accordingly. [1 point]

In [6]:
# data.column
data_wv = data_wv.rename({'Industry': 'Industry',
                          'Size':     'Employee_Size',
                          'OC3':      'Organization_type',
                          'HI1':      'Health_Insurance_1',
                          'HI2':      'Health_Insurance_2',
                          'HI3':      'Health_Insurance_3',
                          'CP1':      'Promoting_Health_at_Work',
                          'WL6':      'WorkLife_Programming_Policies_Benefits',
                          'WD1_1':    'Workforce Demographics_1_1',
                          'WD1_2':    'Workforce Demographics_1_2',
                          'WD2':      'Workforce Demographics_2',
                          'WD3':      'Workforce Demographics_3',
                          'WD4':      'Workforce Demographics_4',
                          'WD5':      'Workforce Demographics_5',
                          'WD6':      'Workforce Demographics_6',
                          'WD7':      'Workforce Demographics_7'}, axis=1)
data_wv.head()

,Industry,Employee_Size,Organization_type,Health_Insurance_1,Health_Insurance_2,Health_Insurance_3,Promoting_Health_at_Work,WorkLife_Programming_Policies_Benefits,Workforce Demographics_1_1,Workforce Demographics_1_2,Workforce Demographics_2,Workforce Demographics_3,Workforce Demographics_4,Workforce Demographics_5,Workforce Demographics_6,Workforce Demographics_7
0,7.0,7.0,3.0,2.0,1.0,2.0,1.0,1.0,25.0,20.0,85.0,60.0,40.0,15.0,0.0,22.0
1,7.0,6.0,3.0,2.0,3.0,1.0,1.0,1.0,997.0,997.0,90.0,90.0,997.0,997.0,0.0,997.0
2,7.0,8.0,3.0,1.0,3.0,1.0,1.0,1.0,35.0,4.0,997.0,997.0,40.0,15.0,997.0,997.0
3,7.0,4.0,2.0,1.0,2.0,1.0,2.0,2.0,50.0,15.0,50.0,85.0,75.0,0.0,0.0,997.0
4,7.0,4.0,3.0,1.0,3.0,1.0,1.0,1.0,50.0,40.0,60.0,60.0,40.0,30.0,0.0,28.0


## Problem 4
Using the codebook and this [dictionary of NAICS industrial codes](https://www.naics.com/search-naics-codes-by-industry/), place descriptive labels on the categories of the industry column in the working data. [1 point]

In [7]:
# Add the rest of the industyr to this map 
replace_map = {1:'Agriculture, Forestry, Fishing and Hunting; etc.', 
               2:'Wholesale Trade', 
               3:'Arts, Entertainment, and Recreation', 
               4:'Information',
               5:'Educational Services',
               6:'Public Administration',
               7:'Hospital',}
data_wv.Industry = data_wv.Industry.map(replace_map)
data_wv.Industry

0                    Hospital
1                    Hospital
2                    Hospital
3                    Hospital
4                    Hospital
                ...          
2838    Public Administration
2839    Public Administration
2840    Public Administration
2841    Public Administration
2842    Public Administration
Name: Industry, Length: 2843, dtype: object

## Problem 5
Using the codebook, recode the "size" column to have three categories: "Small" for workplaces with fewer than 100 employees, "Medium" for workplaces with at least 100 but fewer than 500 employees, and "Large" for companies with at least 500 employees. [Note: Python dataframes have an attribute `.size` that reports the space the dataframe takes up in memory. Don't confuse this attribute with the column named "Size" in the raw data.] [1 point]

In [8]:
data_wv['Employee_Size_Label'] = pd.cut(data_wv.Employee_Size, 
                                        bins=[0,3,5,8], 
                                        labels=("Small", "Medium", "Large"))
data_wv[['Employee_Size_Label']].describe()

,Employee_Size_Label
count,2842
unique,3
top,Small
freq,2195


## Problem 6
Use the codebook to write accurate and descriptive labels for each category for each categorical column in the working data. Then apply all of these labels to the data at once. Code "Legitimate Skip", "Don't know", "Refused", and "Blank" as missing values. [2 points]

In [9]:
# Add more labels to the
# not going to handle WD columns
replace_map = {'Organization_type':{1:'Human Resources and/or Benefits', 
                                    2:'Health Promotion/Fitness/Wellness', 
                                    3:'Safety', 
                                    4:'Medical',
                                    5:'PR/Marketing',
                                    6:'Finance',
                                    7:'Other',
                                    97:np.nan,
                                    99:np.nan,
                                   }}
data_wv = data_wv.replace(replace_map)
data_wv.head()

,Industry,Employee_Size,Organization_type,Health_Insurance_1,Health_Insurance_2,Health_Insurance_3,Promoting_Health_at_Work,WorkLife_Programming_Policies_Benefits,Workforce Demographics_1_1,Workforce Demographics_1_2,Workforce Demographics_2,Workforce Demographics_3,Workforce Demographics_4,Workforce Demographics_5,Workforce Demographics_6,Workforce Demographics_7,Employee_Size_Label
0,Hospital,7.0,Safety,2.0,1.0,2.0,1.0,1.0,25.0,20.0,85.0,60.0,40.0,15.0,0.0,22.0,Large
1,Hospital,6.0,Safety,2.0,3.0,1.0,1.0,1.0,997.0,997.0,90.0,90.0,997.0,997.0,0.0,997.0,Large
2,Hospital,8.0,Safety,1.0,3.0,1.0,1.0,1.0,35.0,4.0,997.0,997.0,40.0,15.0,997.0,997.0,Large
3,Hospital,4.0,Health Promotion/Fitness/Wellness,1.0,2.0,1.0,2.0,2.0,50.0,15.0,50.0,85.0,75.0,0.0,0.0,997.0,Medium
4,Hospital,4.0,Safety,1.0,3.0,1.0,1.0,1.0,50.0,40.0,60.0,60.0,40.0,30.0,0.0,28.0,Medium


## Problem 7
The features that measure the percent of the workforce with a particular characteristic use the codes 997, 998, and 999 to represent "Don't know", "Refusal", and "Blank/Invalid" respectively. Replace these values with missing values for all of the percentage features at the same time. [1 point]

In [10]:
wdcols = [x for x in data_wv.columns if x.startswith("Workforce")]
data_wv[wdcols] = data_wv[wdcols].replace([997, 998, 999], np.nan)
data_wv.head()

,Industry,Employee_Size,Organization_type,Health_Insurance_1,Health_Insurance_2,Health_Insurance_3,Promoting_Health_at_Work,WorkLife_Programming_Policies_Benefits,Workforce Demographics_1_1,Workforce Demographics_1_2,Workforce Demographics_2,Workforce Demographics_3,Workforce Demographics_4,Workforce Demographics_5,Workforce Demographics_6,Workforce Demographics_7,Employee_Size_Label
0,Hospital,7.0,Safety,2.0,1.0,2.0,1.0,1.0,25.0,20.0,85.0,60.0,40.0,15.0,0.0,22.0,Large
1,Hospital,6.0,Safety,2.0,3.0,1.0,1.0,1.0,NaN,NaN,90.0,90.0,NaN,NaN,0.0,NaN,Large
2,Hospital,8.0,Safety,1.0,3.0,1.0,1.0,1.0,35.0,4.0,NaN,NaN,40.0,15.0,NaN,NaN,Large
3,Hospital,4.0,Health Promotion/Fitness/Wellness,1.0,2.0,1.0,2.0,2.0,50.0,15.0,50.0,85.0,75.0,0.0,0.0,NaN,Medium
4,Hospital,4.0,Safety,1.0,3.0,1.0,1.0,1.0,50.0,40.0,60.0,60.0,40.0,30.0,0.0,28.0,Medium


## Problem 8
Sort the working data by industry in ascending alphabetical order. Within industry categories, sort the rows by size in ascending alphabetical order. Within groups with the same industry and size, sort by percent of the workforce that is under 30 in descending numeric order. [1 point]

In [11]:
data_wv.sort_values(by=['Industry', 'Employee_Size', 'Workforce Demographics_1_1'], ascending = [True, True, False])
data_wv.head()

,Industry,Employee_Size,Organization_type,Health_Insurance_1,Health_Insurance_2,Health_Insurance_3,Promoting_Health_at_Work,WorkLife_Programming_Policies_Benefits,Workforce Demographics_1_1,Workforce Demographics_1_2,Workforce Demographics_2,Workforce Demographics_3,Workforce Demographics_4,Workforce Demographics_5,Workforce Demographics_6,Workforce Demographics_7,Employee_Size_Label
0,Hospital,7.0,Safety,2.0,1.0,2.0,1.0,1.0,25.0,20.0,85.0,60.0,40.0,15.0,0.0,22.0,Large
1,Hospital,6.0,Safety,2.0,3.0,1.0,1.0,1.0,NaN,NaN,90.0,90.0,NaN,NaN,0.0,NaN,Large
2,Hospital,8.0,Safety,1.0,3.0,1.0,1.0,1.0,35.0,4.0,NaN,NaN,40.0,15.0,NaN,NaN,Large
3,Hospital,4.0,Health Promotion/Fitness/Wellness,1.0,2.0,1.0,2.0,2.0,50.0,15.0,50.0,85.0,75.0,0.0,0.0,NaN,Medium
4,Hospital,4.0,Safety,1.0,3.0,1.0,1.0,1.0,50.0,40.0,60.0,60.0,40.0,30.0,0.0,28.0,Medium


## Problem 9
There is one row in the working data that has a `NaN` value for industry. Delete this row. Use a logical expression, and not the row number. [1 point]

In [12]:
data_wv.Industry[!data_wv.Industry.isnull()]

SyntaxError: invalid syntax (<ipython-input-12-d7054d9061ca>, line 1)

## Problem 10
Create a new feature named `gender_balance` that has three categories: "Mostly men" for workplaces with between 0% and 35% female employees, "Balanced" for workplaces with more than 35% and at most 65% female employees, and "Mostly women" for workplaces with more than 65% female employees. [1 point]

## Problem 11
Change the data type of all categorical features in the working data from "object" to "category". [1 point]

## Problem 12
Filter the data to only those rows that represent small workplaces that allow employees to work from home. Then report how many of these workplaces offer full insurance, partial insurance, and no insurance. Use a function that reports the percent, cumulative count, and cumulative percent in addition to the counts. [1 point]

## Problem 13
Anything that can be done in SQL can be done with `pandas`. The next several questions ask you to write `pandas` code to match a given SQL query. But to check that the SQL query and `pandas` code yield the same result, create a new database wsing the `sqlite3` package and input the cleaned WHA data as a table in this database. (See module 6 for a discussion of SQlite in Python.) [1 point]

## Problem 14
Write `pandas` code that replicates the output of the following SQL code:
```
SELECT size, type, premiums AS insurance, percent_female FROM whpps
WHERE industry = 'Hospitals' AND premium_change='Smaller'
ORDER BY percent_female DESC;
```
For each of these queries, your feature names might be different from the ones listed in the query, depending on the names you chose in problem 3.
[2 points]

## Problem 15
Write `pandas` code that replicates the output of the following SQL code:
```
SELECT industry, 
    AVG(percent_female) as percent_female, 
    AVG(percent_under30) as percent_under30,
    AVG(percent_over60) as percent_over60
FROM whpps
GROUP BY industry
ORDER BY percent_female DESC;
```
[2 points]

## Problem 16
Write `pandas` code that replicates the output of the following SQL code:
```
SELECT gender_balance, premiums, COUNT(*)
FROM whpps
GROUP BY gender_balance, premiums
HAVING gender_balance is NOT NULL and premiums is NOT NULL;
```
[2 points]